In [1]:
import pandas as pd
import numpy as np
import networkx as nx

TODO: look at this to add more measures
    https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python
        

In [2]:
df=pd.read_excel("./test_data/org.xlsx")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9553 entries, 0 to 9552
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      9553 non-null   int64 
 1   employee_name   9553 non-null   object
 2   employee_id     9553 non-null   int64 
 3   manager_id      9553 non-null   int64 
 4   manager_name    9553 non-null   object
 5   band            9553 non-null   int64 
 6   cost_centre     9553 non-null   object
 7   borg_id         9553 non-null   object
 8   employee_title  9553 non-null   object
 9   is_pa           9553 non-null   bool  
dtypes: bool(1), int64(4), object(5)
memory usage: 681.1+ KB


We create a directed graph from the dataframe.  
Note that we add an attribute "is PA" to the *edge*, we could potentially add it to the node   
The edges can have weights to compute the cost of navigating etc etc, check the library documentation online for more details

In [3]:
G= nx.from_pandas_edgelist(df, 'manager_id', 'employee_id', ['is_pa'],create_using=nx.DiGraph())

In [4]:
for i,r in df.iterrows():
    G.nodes[r['employee_id']]['employee_name'] = r['employee_name']
    G.nodes[r['employee_id']]['band'] = r['band']
    G.nodes[r['employee_id']]['employee_title'] = r['employee_title']
    G.nodes[r['employee_id']]['cost_centre'] = r['cost_centre']
print("Added ", i, " nodes")    
print("Graph G has ", len(G.nodes), " nodes")

Added  9552  nodes
Graph G has  9553  nodes


In [5]:
G.nodes[1]

{'employee_name': 'Steve Jobs',
 'band': 1,
 'employee_title': 'Senior Director',
 'cost_centre': '1.1'}

In [6]:
list(G.successors(1)) # the person AND all the direct reports , neighbors() and successors() are the same

[1, 2, 1814, 2334, 4409, 5449, 6090, 7513]

In [7]:
list(nx.descendants(G,1))[0:19] # these are all reports i.e. the entire team excluding the person

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [8]:
list(nx.ancestors(G,4)) # these are all the bosses, excluding the person itself

[1, 2, 3]

In [9]:
list(G.predecessors(3)) #this is all the direct boss(es), ie having an edge ending in that node. Note that if the node doesnt have any incoming edge, the result is that same node (ie the boss of the top boss is him/herself, i.e. predecedor of root node is that root node.

[2]

In [19]:
#dfs_depth base Code reference networkx
def dfs_depth(G, source=None, depth_limit=None):
    if source is None:
        nodes = G
    else:
        nodes = [source]
    visited = set()
    if depth_limit is None:
        depth_limit = len(G)
    for start in nodes:
        if start in visited:
            continue
        max_depth = 0
        visited.add(start)
        stack = [(start, depth_limit, iter(G[start]))]
        while stack:
            parent, depth_now, children = stack[-1]
            try:
                child = next(children)
                if child not in visited:
                    visited.add(child)
                    if depth_now > 1:
                        if((depth_limit - depth_now + 1)>max_depth):
                            max_depth = depth_limit - depth_now + 1
                        stack.append((child, depth_now - 1, iter(G[child])))
            except StopIteration:
                stack.pop() 
    return max_depth
  



0

Now we are going to create a table with the span of control of each person.  
TODO: look into various other metrics... raw depth, how balanced between managers and staff or maybe between top, medium, low , or between peer nodes (std of direct reports within that remit)

In [24]:
temp=[]
for  n in G.nodes:
    all_leaves=len(list(nx.descendants(G,n)))
    children=len(list(G.successors(n)))-1
    team_depth=dfs_depth(G,n)
    temp.append([n, all_leaves, children, team_depth])
    
df2=pd.DataFrame(temp,columns=['id','full_span','direct_reports','team_depth'])
df2.head()

,id,full_span,direct_reports,team_depth
0,1,9552,7,5
1,2,1811,6,4
2,3,253,6,3
3,4,48,4,2
4,5,7,6,1


In [25]:
nx.ancestors(G,2)

{1}

In [59]:
span_threshold=40
from functools import lru_cache

lru_cache(maxsize=2000)
def find_lower_bound(list_to_check,lower_bound):
    current_result=list_to_check[0]
    for i,e in enumerate(list_to_check):
        if e>=lower_bound:
            if current_result>=lower_bound:
                if e<current_result:
                    current_result=e
            else:
                current_result=e
    if current_result<lower_bound:
        return False
    return list_to_check.index(current_result)

In [67]:
for n in list(G.nodes):
    a_list=list(nx.ancestors(G,n))
    if a_list ==[] :
        G.nodes[n]["first_boss"]=n
        continue
    a_span=[]
    for a in a_list:
        span=len(nx.descendants(G,a))
        a_span.append(span)   
    first_boss_index=find_lower_bound(tuple(a_span),span_threshold)
    if first_boss_index:
        G.nodes[n]["first_boss"]=a_list[first_boss_index]